# Regression (40 points).

**Train a Regression model in which: the independent variables (inputs) are
“ASR_Error,” “IntentError,” “Duration,” “Gender,” and “Purchase”; the dependent
variable (output) is “SUS., and anwer the following questions:**

In [1]:
!pip install seaborn


[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Python Lib

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# Read Data

In [3]:
df = pd.read_csv('data.csv')
df = df.fillna(0)
df.head(10)

,Purchase,SUS,Duration,Gender,ASR_Error,Intent_Error,Unnamed: 6
0,1,84,254,0,3,2,0.0
1,0,58,247,0,6,9,0.0
2,0,56,125,1,6,8,0.0
3,0,55,22,0,11,7,0.0
4,1,95,262,0,2,3,0.0
5,0,71,113,1,8,9,0.0
6,0,76,91,1,6,3,0.0
7,0,64,46,0,6,11,0.0
8,0,56,132,0,9,9,0.0
9,0,96,190,0,11,4,0.0


In [4]:
df.isna().sum()

Purchase        0
SUS             0
Duration        0
Gender          0
ASR_Error       0
Intent_Error    0
Unnamed: 6      0
dtype: int64

# Five of our variables, Purchase, Duration, Gender, ASR_Error, and Intent_Error, are categorical; we need to encode them so that our regression models can take their values as input

In [5]:
! pip install -U scikit-learn


[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()

le.fit(df['Purchase'])
df['Purchase'] = le.transform(df['Purchase'])

le.fit(df['Gender'])
df['Gender'] = le.transform(df['Gender'])

le.fit(df['Duration'])
df['Duration'] = le.transform(df['Duration'])

le.fit(df['ASR_Error'])
df['ASR_Error'] = le.transform(df['ASR_Error'])

le.fit(df['Intent_Error'])
df['Intent_Error'] = le.transform(df['Intent_Error'])

df.head(10);

# Let's start with regression analysis

In [35]:
!pip install statsmodels

     ---------------------------------------- 9.0/9.0 MB 13.7 MB/s eta 0:00:00
     ------------------------------------- 233.8/233.8 kB 14.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import statsmodels.api as sm

y = df['SUS'] # dependent variable
x = df.drop(columns='SUS') # predictor variables

In [37]:
#add constant to predictor variables
x = sm.add_constant(x)

model = sm.OLS(y, x).fit()
print(model.summary());

                            OLS Regression Results                            
Dep. Variable:                    SUS   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     27.40
Date:                Tue, 14 Feb 2023   Prob (F-statistic):           5.22e-17
Time:                        17:09:15   Log-Likelihood:                -362.39
No. Observations:                 100   AIC:                             736.8
Df Residuals:                      94   BIC:                             752.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           92.8514      5.526     16.802   

C:\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1934: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error

In [39]:
x = df.drop(columns='SUS')
y = df['SUS']

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [41]:
lr = LinearRegression().fit(x_train,y_train)

y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print("The R square score of linear regression model is: ", lr.score(x_test,y_test))

The R square score of linear regression model is:  0.6174625725418394


In [51]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=40)
#test size of 0.2 is commonly accepted.
#random state number was chosen randomly and serves as a constant.

In [52]:
X_train = sm.add_constant(X_train) # add constant term to model
reg = sm.OLS(y_train, X_train)
result = reg.fit()

In [54]:
X_test = sm.add_constant(X_test)
y_pred = result.predict(X_test)
r_squared = result.rsquared
mse = np.mean((y_test - y_pred)**2)
mae = np.mean(abs(y_test - y_pred))

# interpret results

In [57]:
coef_df = pd.DataFrame({"variable": x.columns, "coefficient":  result.params[1:], "p-value": result.pvalues[1:]})
print(coef_df)
print("r-squared:", r_squared)
print("Mean squared error:", mse)
print("Mean absolute error:", mae)

                  variable  coefficient   p-value
Purchase          Purchase     1.817327  0.636308
Duration          Duration     0.029839  0.473183
Gender              Gender     2.294973  0.286864
ASR_Error        ASR_Error    -1.452559  0.001659
Intent_Error  Intent_Error    -1.896366  0.000101
Unnamed: 6      Unnamed: 6     0.000000       NaN
r-squared: 0.5934753604201755
Mean squared error: 108.35216959281408
Mean absolute error: 7.705552463424361


# 1.What features are significant? What features are insignificant?

In the table of coefficients and p-values, the significant features are those with p-values less than the significance level (often set at 0.05). In this case, ASR_Error and Intent_Error have p-values of 0.001659 and 0.000101 respectively, indicating they are significant.

The features Purchase, Duration, and Gender have p-values greater than the significance level, indicating that they are not significant.

# 2.Were the results what you expected? Explain why or why not, for each feature.

I expected lower SUS scores as ASR_Error and Intent Error increased. Based on the coefficient values, we can see that ASR_Error and Intent_Error have a negative relationship with SUS scores, meaning that as the number of ASR errors or intent classification errors increase, SUS scores tend to decrease.

I did not know whether Gender would have an impact on the SUS scores, but the results show male users tend to lead to higher SUS scores. The coefficient for the Gender variable in the model indicates that, on average, male users had higher SUS scores than female users, but the p-value for that coefficient suggests that the difference between genders is not statistically significant at the conventional level of significance.

I expected longer Duration to lead to higher SUS scores because it could be a sign of a positive interaction with Siri. Duration had a positive relationship with SUS scores, longer interactions with Siri tend to lead to higher SUS scores.

I did not expect Purchase to be an indicator of higher SUS scores. Since Purchase had a higher P-Value, this suggests that purchasing a ticket using Siri does not significantly affect the SUS scores.

# 3.What does the model suggest is the most influential factor on SUS? Explain what tells you this is the most influential factor statistically.

The model suggests that the most influential factor on SUS is the variable "Intent_Error". This can be seen from the coefficient for "Intent_Error", which is the largest negative coefficient in the model (-1.896366), meaning that higher values of Intent_Error are associated with lower SUS scores.

The p-value for Intent_Error is very small (0.000101), meaning that this relationship is statistically significant.

# 4.What are the potential reasons for these factor(s) being significant predictors of SUS?

Based on the coefficients from the results, it appears that errors made by Siri, such as ASR and Intent classification errors, have a negative effect on SUS scores, meaning that higher error rates lead to lower SUS scores. This means that accurate voice recognition and intent classification are essential for users to have a positive experience with Siri when purchasing flight tickets.